In [29]:
module RobotWorld

    using Plots

    mutable struct World
        objects::Array{World}
        World() = new()
    end

    function append(self::World, obj::World)
        push!(self.objects, deepcopy(obj))
    end

    function draw(self::World)

        # initialize a plot
        plt = plot(
            1,
            label="trajectory", 
            aspectratio=1,
            xlim=(-5, 5),
            ylim=(-5, 5),
            xlabel="X", 
            ylabel="Y"
        )

        for obj in self.objects
            draw(obj)
        end
    end

end

LoadError: invalid redefinition of constant RobotWorld

In [27]:
using .RobotWorld


LoadError: invalid using path: "RobotWorld" does not name a module